함수형 프로그래밍은 일급 계층 객체로서의 함수를 강조한다. 함수를 인자로 받거나 함수를 결과 값으로 반환하는 많은 고차 함수를 살펴봤다. 이번 장에서는 함수의 생성과 변경을 돕는 여러 함수가 들어 있는 functools 라이브러리를 살펴본다.

functools 모듈에 있는 함수 중 다음과 같은 것을 살펴본다.

* @lru_cache: 애플리케이션 유형에 따라 이 데코레이터를 사용하면 상당한 성능 향상을 얻을 수 있다.

* @totla_ordering: 이 데코레이터는 다양한 비교 연산자를 만들 수 있도록 돕는다. 이를 통해 객체지향 설계와 함수형 프로그래밍을 어떻게 혼합할 수 있는지에 대한 좀 더 일반적인 질문을 살펴본다.

* partial(): 어떤 기존 함수의 인자 중 일부만을 적용한 새로운 함수를 만든다.

* reduce(): sum()과 같은 축약을 만드는 것을 일반화한 고차 함수다.

### lru_cache로 결과 캐시하기

lru_cache 데코레이터는 주어진 함수를 더 빨리 작동할 수 있는 함수로 바꿔준다. LRU는 최소 최근 사용을 뜻한다. 이러한 방식에서는 최근에 사용한 원소를 풀에 일정하게 유지하다가 풀이 가득찬 경우, 최근에 가장 덜 자주 사용한 원소를 버려 풀이 넘치는 것을 방지한다.

이 함수는 데코레이터다. 따라서 계산 결과를 캐시에 남겨 성능이 향상될 가능성이 있는 모든 함수에 이를 적용할 수 있다. 사용법은 다음과 같다.

In [1]:
from functools import lru_cache
@lru_cache(128)
def fibc(n):
    if n == 0: return 0
    if n == 1: return 1
    return fibc(n-1) + fibc(n-2)

캐시에 n이 있다면 비용이 많이 들 수도 있는 계산을 다시 하지 않고, 캐시에 있는 예전에 계산했던 결과를 반환한다. 각 반환 값은 캐시에 추가된다. 캐시가 가득차면, 가장 오래된 값을 없애 새로운 값이 들어갈 공간을 만든다.

재귀를 조심성 없이 구현하면, 비용이 꽤 많이 든다. 전제 복잡도는 O(n^2)이다.

In [2]:
fibc(20)

6765

In [3]:
fibc(200)

280571172992510140037611932413038677189525

In [4]:
fibc(1000)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

In [13]:
fibc(5000)

3878968454388325633701916308325905312082127714646245106160597214895550139044037097010822916462210669479293452858882973813483102008954982940361430156911478938364216563944106910214505634133706558656238254656700712525929903854933813928836378347518908762970712033337052923107693008518093849801803847813996748881765554653788291644268912980384613778969021502293082475666346224923071883324803280375039130352903304505842701147635242270210934637699104006714174883298422891491273104054328753298044273676822977244987749874555691907703880637046832794811358973739993110106219308149018570815397854379195305617510761053075688783766033667355445258844886241619210553457493675897849027988234351023599844663934853256411952221859563060475364645470760330902420806382584929156452876291575759142343809142302917491088984155209854432486594079793571316841692868039545309545388698114665082066862897420639323438488465240988742395873801976993820317174208932265468879364002630797780058759129671389634214252579116872755600360311370

이렇게 메모이제이션이라는 개념은 강력하다. 결과를 메모이제이션하면 성능이 향상되는 알고리즘도 많다. 하지만 일부 알고리즘은 그리 큰 이륵을 보지 못하기도 한다. 

비슷한 값을 여러 번 계산하는 프로그램이라면 성능을 향상시킬 수있다. 하지만 캐시에 넣은 값을 재활용하는 일이 적다면, 캐시를 유지하는 데 드는 비용이 성능 향상보다 더 커져 버린다.

이항 계수를 lru_cache로 다루면 캐시를 초기화하는 연산에 따른 비용이 더 들어가 실제보다 성능이 더 나쁘게 나타난다. lru_cache 데코레이터를 추가하는 것은 매우 쉽지만, 그 결과는 심오할 수도 있고 실제 데이터의 분포화 알고리즘에 따라서는 아무런 영향이 없을 수도 있다는 것이다. 

따라서 캐시가 상태가 있는 객체라는 사실을 알아두는 것이 중요하다. 이 설계는 순수 함수형 프로그래밍의 한계를 넘어선다. 이상적인 것은 대입문을 없애고 그에 따른 상태 변화를 피하는 것이다. 이러한 상태 변화를 피하는 것의 전형적인 예가 바로 재귀함수다. 재귀함수에서는 현재 상태가 함수의 인자 값에 들어있지, 어떤 변수에 들어 있지 않다. 하지만 이렇게 이상화한 재귀가 실제 프로세서 하드웨어와 메모리에 제약하에서 효율적으로 작동하려면 꼬리재귀호출 회적화가 필수적이라는 것을 살펴봤다. 파이썬에서는 꼬리재귀를 for 루프로 바꿔 수동으로 최적화할 수 있다. 캐시도 필요할 때마다 수동으로 도입할 수 잇는 비슷한 종류의 최적화다.

### 완전한 순서가 정해져 있는 클래스 정의하기

total_ordering 데코레이터는 다양한 비교 연산자를 제공하는 새로운 클래스를 정의할 때 도움이 된다. 이 데코레이터를 numbers.Number의 하위 클래스인 수를 표현하는 클래스에 적용할 수 있다. 또한 절반 정도의 수와 비슷한 클래스에도 적용할 수 있다.

수와 비슷한 크랠스의 예를 살펴보기 위해 트럼프 카드를 생각해보자. 카드에는 끝 수와 슈트가 있다. 몇몇 게임에서는 끝 수가 특히 중요하다. 수와 마찬가지로 카드에도 순서가 정해져 있다. 떄로 각 카드를 마치 수인 것처럼 다루면서, 점수의 합계를 계산하곤 한다. 

실제로는 카드를 끝 수만으로 비교하는 편이 더 낫다. 다음은 훨씬 유용한 클래스 정의다. 이 정의를 두 가지 부분으로 나눠 설명할 것이다. 첫 번째 부분에는 필수적인 애트리뷰트들이 들어 있다.

In [41]:
from functools import total_ordering
from numbers import Number

@total_ordering
class Card(tuple):
    __slots__ = ()
    def __new__(class_, rank, suit):
        obj = tuple.__new__(Card, (rank, suit))
        return obj
    def __repr__(self):
        return "{0.rank}{0.suit}".format(self)
    
    @property
    def rank(self):
        return self[0]
    
    @property
    def suit(self):
        return self[1]
    
    def __eq__(self, other):
        if isinstance(other, Card):
            return self.rank == other.rank

        elif isinstance(other, Number):
            return self.rank == other
        
    def __lt__(self, other):
        if isinstance(other, Card):
            return self.rank < other.rank
        
        elif isinstance(other, Number):
            return self.rank < other

이 클래스는 tuple을 확장한다. 아무런 슬롯도 추가하지 않기 때문에 이 클래스도 변경 불가능한 클래스다. \__new__ 메서드를 재정의(override)하여 끝 수와 슈트를 지정할 수 있게 한다. 또한 \__repr__ 메서드를 통해 Card 객체의 문자열 표현을 제공한다. 또한 끝 수와 슈트를 애트리뷰트 이름을 사용해 얻을 수 있도록 두가지 프로퍼티를 추가했다.

이 클래스 정의의 나머지 부분은 단지 두 가지 비교 연산만을 정의하는 것을 보여준다.

\__eq__와 \__lt__ 함수를 정의했다. total_ordering 데코레이터가 나머지 모든 비교 연산을 만들어준다. 두 함수 모두 카드와 카드의 비교나 카드와 수의 비교가 가능하다.

우선, 다음과 같이 끝수만을 사용하여 비교를 제대로 수행할 수 있다.

In [44]:
c2s = Card(2, '\u2660')
c2h = Card(2, '\u2665')

c2h == c2s

True

In [45]:
c2h == 2

True

이 클래스를 사용하면, 여러 시뮬레이션에서 카드의 끝 수를 비교하는 구문을 단순화 할 수 있다. 더 나아가 다음과 같이 다양한 비교 연산을 사용할 수 있다.

In [46]:
c2s = Card(2, '\u2660')
c2h = Card(3, '\u2665')
c2c = Card(4, '\u2663')

c2s <= c2h < c2c

True

In [49]:
c2h >= c2h

True

In [52]:
c2h > c2s

True

In [53]:
c2c != c2s

True

모든 비교 함수를 직접 작성할 필요가 없고, 데코레이터가 이 함수들을 만들어준다. 데코레이터가 만들어주는 연산자들이 완벽하지는 않다. 정수와의 비교와 Card 인스턴스와의 비교를 모두 사용하도록 요구했다. 그에 따라 몇 가지 문제가 드러난다.

In [54]:
c2c > 3

TypeError: '>' not supported between instances of 'Card' and 'int'

In [55]:
3 < c2c

TypeError: '<' not supported between instances of 'int' and 'Card'

### 수 클래스 정의하기

어떤 경우에는 파이썬으로 사용할 수 있는 수 타입의 탑을 확장하고 싶을 때가 있다. numbers.Number의 하위 클래스를 사용하면 함수형 프로그램을 단순하게 만들 수 있는 경우가 있다. 예를 들어, 복잡한 알고리즘의 일부를 Number의 하위 클래스 정의 포함시킴으로써, 애플리케이션의 다른 부분을 좀 더 깔끔하고 단순하게 만들 수 있다.

파이썬은 이미 다양한 수 타입을 제공하고 있다. int와 float 내장타입을 다양한 문제에 활용할 수 있다. decimal.Decimal 패키지는 금전을 다루는 경우를 잘 지원할 수 있다. 어떤 경우에는 float 변수보다 fractions.Fraction 클래스가 더 쓸모 있다는 것을 알게 될 수도 있다.

파이썬의 Number 클래스는 변경 불가능한 객체로 사용하려는 의도로 만들어졌으며, 일반적인 함수형 설계를 모든 메서드 함수에 적용할 수 있다. 예외적인 파이썬 내부 특별 메서드(\__iadd__)는 그냥 무시하면 된다.

Number의 하위 클래스를 사용하는 경우, 다음과 같이 설계 시 고려해야 할 내용이 매우 많다.

* 동등성 검사와 해시 값 계산, 수에 대한 해시 계산에 있어 핵심적인 특징은 파이썬 표준 라이브러리의 9.1.2절 타입을 구현하는 사람을 위한 주의사항을 보면 정리되어 있다.

* 다른 비교 연산자들.

* 산순 연산자인 +, -, *, /, //, %, \***. 일반적인 연산자에 대한 특별 메서드뿐만 아니라 역방향 타입 일치를 위한 추가 메서드도 존재한다. 예를 들어 a - b와 같은 식이 있다면 파이썬은 a.\__sub__(b) 메서드 함수의 구현을 찾는다. 결과적으로, a.\__sub__(b) 메서드를 호출한다. 만약, 좌항인 a의 클래스에 해당 메서드가 없거나 NotImplemented 예외가 발생한다면 우항의 값에서 결과를 만들어 낼 수 있는 b.\__rsub__(a)라는 메서드를 찾을 수 있는지도 살펴본다. b의 클래스가 a의 클래스의 하위 클래스인 경우를 처리하는 특별한 규칙도 존재한다. 이 규칙을 사용하면 하위 클래스가 상위 클래스의 좌항 연산자 처리 규칙을 마음대로 바꿀 수 있다.

* 비트 처리 연산자인 &, |, ^, >>, <<, ~. 이러한 연산자는 부동 소수점 수의 경우 의미가 없다. 따라서 그러한 경우에는 이러한 연산에 대한 특별 메서드를 제외하는 게 가장 좋은 설계일 것이다.

* round(), pow(), divmod()와 같은 추가 함수도 수에 대한 특별 메서드 이름을 사용해 정의할 수 있다. 이들은 각 하위 클래스 따라 적절한 구현을 제공해야 한다.

### partial()로 인자 중 일부만 적용하기

partial() 함수는 부분 적용이라고 부르는 동작을 가능하게 한다. 부분 적용한 함수는 기존 함수와 그 함수가 요구하는 인자 중 일부분을 가지고 만든 새로운 함수다. 이는 커링과 밀접한 관계가 있다. 파이썬의 함수 구현 방식에는 커링을 적용할 수 없기 때문에 여기서는 이론적인 배경 지식 중 대부분은 그리 중요하지 않다. 하지만 커링이라는 개념 자체는 우리에게 몇 가지 편리한 단순화 방법을 제공한다.

다음과 같은 간단한 예제를 살펴보자.

In [56]:
from functools import partial

exp2 = partial(pow, 2)

In [57]:
exp2(12)

4096

In [58]:
exp2(17) -1

131071

pow(2, y)를 호출하는 exp2(y)라는 함수를 만들었다. partial() 함수는 첫 번째 위치에 있는 매개변수를 pow() 함수에 연결시켜준다. 이렇게 만든 exp2() 함수를 호출하면, partial() 함수로 연결한 인자와 exp2() 함수를 통해 받은 인자를 함께 pow2()에 전달한 결과를 돌려받는다. 

위치 매개변수를 연결하는 것은 엄격하게 왼쪽에서 오른쪾 순서를 따른다. 키워드 매개변수를 받는 함수의 경우, 부분 적용한 함수를 만들 때 키워드를 넘길 수도 있다.

이러한 종류의 부분 적용 함수는 람다를 사용해 다음과 같은 방식으로도 만들 수 있다.

In [59]:
exp2 = lambda y: pow(2, y)

어느 쪽도 확실히 더 나은 방법은 없다. 성능을 비교해보면 partial 함수가 람다보다 종이 한 장만큼 더 빠르다. 1,000,000번 반복에 대해 0.05초의 차이를 보인다. 따라서 그리 큰 성능 향상은 없다. 

람다가 partial()의 모든 기능을 제공하는 것 같기 때문에 이 함수를 그리 유용하지 않다고 생각하고 무시한다고 해도 안전해 보인다. 커링을 사용해 부분 적용 함수를 만드는 방법에 대해 추후에 살펴본다.

### 데이터 집합을 reduce()를 사용해 축약하기

sum(), len(), min() 함수는 모두 reduce() 함수로 표현할 수 있는 좀 더 일반적인 알고리즘을 구체화시킨 것들이다. reduce() 함수는 반복 가능 객체의 원소 쌍에 대해 함수를 겹쳐 적용하는 고차 함수다. 

다음과 같은 시퀀스 객체가 있다고 가정하자.

In [61]:
d = [2, 4, 4, 4, 5, 5, 7, 9]

from functools import reduce
reduce(lambda x, y : x+y ,d)

40

파이썬의 식은 왼쪽에서 오른쪽 순서로 연산자를 적용한다. 따라서 이렇게 왼쪽에서 오른쪽으로 겹쳐 나가는 것은 의미에 큰 변화를 주지 않는다.

### map과 reduce 조합하기

간단한 정의를 통해 고차 함수를 정의하는 방법을 알 수 있다.

In [62]:
def map_reduce(map_fun, reduce_fun, iterable):
    return reduce(reduce_fun, map(map_fun, iterable))

map과 reduce를 사용해 제곱의 합을 계산하는 축약을 다음과 같이 정의할 수 있다.

In [63]:
def sum2_mr(iterable):
    return map_reduce(lambda y: y**2, lambda x, y: x+y, iterable)

In [65]:
import operator

def sum2_mr2(iterable):
    return map_reduce(lambda y: y**2, operator.add, iterable)

다음은 반복 가능 객체의 원소 개수를 세는 프로그램이다.

In [66]:
def count_mr(iterable):
    return map_reduce(lambda y:1, operator.add, iterable)

이렇게 일반적인 reduce 함수를 사용하면 원하는 대규모 데이터 집합을 단일 값으로 축약하는 연산을 원하는 대로 지정할 수 있다. 하지만 이 reduce() 함수로 수행할 수 있는 것에는 몇 가지 제약이 존재한다.

다음과 같은 명령을 가급적 피해야 한다.

In [67]:
reduce(operator.add, ["1", ",", "2", ",", "3"], "")

'1,2,3'

이 코드는 잘 작동한다 하지만 "".join(["1", ",", "2", ",", "3"])과 같은 식으로 하는 것이 좀 더 효율적이다. 약 3배 정도 성능의 차이가 난다.

In [68]:
"".join(["1", ",", "2", ",", "3"])

'1,2,3'

### 원데이터를 정리하기 위해 map과 reduce 사용하기

In [69]:
def comma_fix(data):
    try:
        return float(data)
    except ValueError:
        return float(data.replace(",",""))

def clean_sum(cleaner, data):
    return reduce(operator.add, map(cleaner, data))

간단한 매핑으로 데이터를 부동 소수점 수로 사용할 수 있는 대부분 올바른 형식으로 변환한다.

앞에서 정의한 함수를 다음과 같이 적용할 수 있다.

In [70]:
d = ['1,196', '1,176', '1,269', '1,240', '1,307']
clean_sum(comma_fix, d)

6188.0

### groupby와 reduce 사용하기

데이터를 분할해 여러 그룹으로 나눈 다음 각각을 요약하는 경우도 자주 있다. defaultdict(list) 방식응 사용해 데이터를 분할할 수 있다. 그 후 각각의 그룹을 따로따로 처리할 수 있다. 

In [71]:
from collections import defaultdict

def partition(iterable, key = lambda x:x):
    pd = defaultdict(list)
    for row in iterable:
        pd[key(row)].append(row)
    for k in sorted(pd):
        yield k, iter(pd[k])

이렇게 하면 반복 가능 객체에 속한 각 원소를 개별 그룹에 분리해 넣는다. 이 함수의 결과 값은 itertools.groupby 함수의 결과와 같다. 

다음은 같은 기능을 itertools.groupby()로 구현한 것이다.

In [73]:
from itertools import groupby

def partition_s(iterable, key = lambda x:x):
    return groupby(iterable, key)

차이점은 groupby 함수에서 사용할 반복 가능 객체는 미리 키를 기준으로 정렬돼 있어야 하는 반면, defaultdict을 사용하는 경우에는 정렬하지 않다로 된다는 점에 있다. 